In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-08-26 11:05:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-08-26 11:05:44 (39.9 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [2]:
! wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

--2021-08-26 11:05:45--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110 (2.1K) [text/x-python]
Saving to: ‘spacex_dash_app.py’

spacex_dash_app.py  100%[===================>]   2.06K  --.-KB/s    in 0s      

2021-08-26 11:05:46 (23.6 MB/s) - ‘spacex_dash_app.py’ saved [2110/2110]



In [3]:
! pip install dash 
! pip install jupyter_dash

     |████████████████████████████████| 1.1 MB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 3.7 MB 74.5 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 5.0 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 66.2 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 73.6 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 69.4 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl size=1100478 sha256=30acebe0cd38adaa5d5251f3208eba8e5bcf858f7669bec557c3296c7b31303e
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/56/e4/21/8a1190a7c31f3aa3f8834b05126ded8bf105b8fd11fad1806e
  Created wheel for dash-core-components: filename=dash_core_components-1.17.1-py3-none-any.whl size=3738983 sha256=b205f0ebc7cc0d29df285a6af745dad8e9eb03f8b06310302a0b5b5285a009bd
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/08/ae/6a/0f4a0ee8fa216e

In [5]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import numpy as np


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

dropdown_options= [{"value":"SITE{}".format(i), "label":v} for i,v in enumerate(spacex_df["Launch Site"].unique())]+\
                [{"value":"ALL_SITES", 'label': 'All Sites'}]
sites_dic = {"SITE{}".format(i):v for i,v in enumerate(spacex_df['Launch Site'].unique())}

# Create an app layout
app.layout = html.Div(children=[
                                html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
    
                                dcc.Dropdown(id='site-dropdown',
                                            options=dropdown_options,
                                            placeholder='Select a Launch Site here',
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks={str(k):str(k) for k in np.arange(0, 10000, 1000)},
                                                value=[2000],
                                               ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
            Output(component_id='success-pie-chart', component_property="figure"),
            Input(component_id='site-dropdown', component_property='value'))
def callback_dropdown(site):
    if site == 'ALL_SITES':
        fig = px.pie(data_frame=spacex_df.groupby("Launch Site")["class"].mean().reset_index(), names='Launch Site', values='class')
        return fig
    elif site in sites_dic.keys():
        data=spacex_df[spacex_df["Launch Site"] == sites_dic[site]]["class"].value_counts(normalize=True).reset_index()
        
        fig = px.pie(data_frame=data, names='index', values='class')
        return fig
    else:
        fig = px.pie(data_frame=spacex_df.groupby("Launch Site")["class"].mean().reset_index(), names='Launch Site', values='class')
        return fig
        
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
        Output(component_id='success-payload-scatter-chart', component_property='figure'),
        Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")
    )
def update_scatter(site, thresh):
    if site == 'ALL_SITES':
        fig = px.scatter(data_frame=spacex_df[(spacex_df["Payload Mass (kg)"]<thresh[0])], x='Payload Mass (kg)', y='Launch Site', color="class")
        return fig
    elif site in sites_dic.keys():
        

        data=spacex_df[(spacex_df["Launch Site"] == sites_dic[site]) &
                       (spacex_df["Payload Mass (kg)"]<thresh[0])]
        
        fig = px.scatter(data_frame=data, x='Payload Mass (kg)', y='class', color="Booster Version")
        
        return fig
    else:
        fig = px.scatter(data_frame=spacex_df[(spacex_df["Payload Mass (kg)"]<thresh[0])], x='Payload Mass (kg)', y='Launch Site', color="class")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(host="localhost", mode="inline")


/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()
